# Getting Started with LangGraph

A quick overview of building an agentic workflow from start to finish.

## Learning Objectives

By the end of this notebook, you will:

1. **Setup your environment** - Load API credentials and prepare your development environment
2. **Define tools** - Create tools that your LLM can invoke (currency converter and EMI calculator)
3. **Initialize the LLM** - Set up the language model and bind tools to it
4. **Define graph nodes** - Create the LLM node and router function for decision-making
5. **Build the graph** - Assemble nodes and edges into a complete workflow
6. **Test your agent** - Validate the graph works with a sample query

This notebook provides a complete end-to-end overview. Subsequent notebooks will dive deeper into each component.

## Step 1: Setup Environment Variables

In [ ]:
# Load environment variables (make sure you have .env with GOOGLE_API_KEY)
from dotenv import load_dotenv

load_dotenv("../../.env")
print("✅ Environment loaded")

## Step 2: Define Tools

In [ ]:
from langchain_core.tools import tool

In [ ]:
# Define currency converter tool
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}"
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

In [ ]:
# Define EMI calculator tool
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

## Step 3: Initialize LLM and Bind Tools

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

In [ ]:
# Create tools list
tools = [currency_converter, emi_calculator]

In [ ]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)
print("✅ LLM initialized with tools")

## Step 4: Define Graph Nodes

In [ ]:
from langgraph.graph import MessagesState, END
from typing import Literal

In [ ]:
# Define LLM node
def call_llm(state: MessagesState):
    """LLM node that invokes the LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

print("✅ LLM node defined")

In [ ]:
# Define router function
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router that decides whether to use tools or end execution."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

print("✅ Router function defined")

## Step 5: Build the Graph

In [ ]:
# Import graph components
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START
from langgraph.prebuilt import ToolNode

In [ ]:
# Build the graph
workflow = StateGraph(MessagesState)

# Add nodes
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))

# Add edges
workflow.add_edge(START, "llm")
workflow.add_conditional_edges(
    "llm",
    should_continue,
    {"tools": "tools", END: END}
)
workflow.add_edge("tools", "llm")

In [ ]:
# Compile the graph into an executable application
app = workflow.compile()
print("✅ Graph compiled")

## Step 6: Test the Graph

In [ ]:
# Create a test query
test_state = {
    "messages": [HumanMessage(content="Convert 100 USD to EUR")]
}

In [ ]:
# Execute the graph
result = app.invoke(test_state)

In [ ]:
# View the final response
result["messages"][-1].content

## Conclusion

Congratulations! You've built your first LangGraph agentic workflow.

### What You Accomplished

In this notebook, you:

✅ **Set up your environment** - Loaded API credentials using environment variables

✅ **Defined tools** - Created a currency converter and EMI calculator that the LLM can invoke

✅ **Initialized the LLM** - Set up Google Gemini and bound your tools to it

✅ **Defined graph nodes** - Created an LLM node and router function for intelligent decision-making

✅ **Built the graph** - Assembled nodes and edges to create a complete agentic workflow

✅ **Tested your agent** - Successfully executed a currency conversion query

### How It Works

Your agent follows this workflow:

1. **User Query** → The graph receives your message
2. **LLM Decision** → The LLM analyzes the query and decides to call the currency_converter tool
3. **Tool Execution** → The tool runs and returns the conversion result
4. **LLM Synthesis** → The LLM formats the result into a natural language response
5. **Final Response** → You receive: "100 USD is approximately equal to 92 EUR."

### Next Steps

This was a quick overview. The following notebooks will dive deeper into each component:

- **Notebook 02**: Environment setup and validation in detail
- **Notebook 03**: Understanding the `@tool` decorator
- **Notebooks 04-05**: Building practical tools step-by-step
- **Notebook 06**: Graph construction fundamentals
- **Notebooks 07-10**: Advanced execution patterns (single, parallel, sequential, conversational)

Continue to **Notebook 02: LangGraph Setup & Validation** to begin your deep dive! →